Here is the case study for Lending Club data analysis. 
The data given below contains information about past loan applicants and whether they ‘defaulted’ or not. The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [95]:
lc = pd.read_csv('loan.csv')

C:\Users\venkats\AppData\Local\Temp\ipykernel_16064\3295163602.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  lc = pd.read_csv('loan.csv')


In [96]:
lc.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [105]:
lc.inq_fi.unique()

array([nan])

In [101]:
loan = lc[['id','annual_inc','loan_status','loan_amnt','emp_length','funded_amnt', 'funded_amnt_inv', 'term','issue_d','int_rate','installment', 'grade','sub_grade','dti','delinq_2yrs','home_ownership', 'revol_util', 'purpose','verification_status']]

In [102]:
# Check the remaining fields dataframe its Information

loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   39717 non-null  int64  
 1   annual_inc           39717 non-null  float64
 2   loan_status          39717 non-null  object 
 3   loan_amnt            39717 non-null  int64  
 4   emp_length           38642 non-null  object 
 5   funded_amnt          39717 non-null  int64  
 6   funded_amnt_inv      39717 non-null  float64
 7   term                 39717 non-null  object 
 8   issue_d              39717 non-null  object 
 9   int_rate             39717 non-null  object 
 10  installment          39717 non-null  float64
 11  grade                39717 non-null  object 
 12  sub_grade            39717 non-null  object 
 13  dti                  39717 non-null  float64
 14  delinq_2yrs          39717 non-null  int64  
 15  home_ownership       39717 non-null 

In [114]:
# find the percentage of nulls in the columns
loan[lc.loan_status =='Charged Off' ].isnull().sum()/len(lc)*100


id                     0.000000
annual_inc             0.000000
loan_status            0.000000
loan_amnt              0.000000
emp_length             0.000000
funded_amnt            0.000000
funded_amnt_inv        0.000000
term                   0.000000
issue_d                0.000000
int_rate               0.000000
installment            0.000000
grade                  0.000000
sub_grade              0.000000
dti                    0.000000
delinq_2yrs            0.000000
home_ownership         0.000000
revol_util             0.040285
purpose                0.000000
verification_status    0.000000
dtype: float64

In [113]:
# Since about .5% of EMp_length was null, impute with maximum occurence values. Here data is filtered with loan_status
loan.loc[(lc.loan_status =='Charged Off') & (lc.emp_length.isnull()), 'emp_length' ]= '10+ yearas'

Remove the rows corresponding to current loan data as they are still paying loan and might not have insights about will they be defaulter or complete the loan

In [63]:
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [69]:
#Handling Other limited null values

lc.emp_length.value_counts()

emp_length
10+ years    8488
< 1 year     4508
2 years      4291
3 years      4012
4 years      3342
5 years      3194
1 year       3169
6 years      2168
7 years      1711
8 years      1435
9 years      1226
Name: count, dtype: int64

In [73]:
lc.emp_length.describe()

count         37544
unique           11
top       10+ years
freq           8488
Name: emp_length, dtype: object

In [75]:
# find the people who defaulted with grade A and find when loan was granted

lc[(lc.loan_status =='Charged Off') & (lc.grade =='A')]

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
46,1069465,5000,5000,5000.000000,36 months,8.90%,158.77,A,A5,10+ years,...,158.77,NaN,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
137,1068111,11100,11100,11100.000000,36 months,6.62%,340.82,A,A2,10+ years,...,340.82,NaN,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
168,1067644,2500,2500,2500.000000,36 months,7.51%,77.78,A,A3,NaN,...,77.78,NaN,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
224,1066766,7200,7200,7200.000000,36 months,7.90%,225.29,A,A4,3 years,...,225.29,NaN,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
266,1066162,12000,12000,12000.000000,36 months,8.90%,381.04,A,A5,7 years,...,381.04,NaN,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39363,218331,10000,10000,1374.997237,36 months,7.75%,312.22,A,A3,6 years,...,640.05,NaN,0.0,1,INDIVIDUAL,0,0.0,0,NaN,0.0
39393,212894,3250,3250,1899.997337,36 months,7.75%,101.47,A,A3,4 years,...,101.47,NaN,0.0,1,INDIVIDUAL,0,0.0,0,NaN,0.0
39406,209349,7200,7200,750.000000,36 months,8.07%,225.86,A,A4,10+ years,...,225.86,NaN,0.0,1,INDIVIDUAL,0,0.0,0,NaN,0.0
39410,207910,2225,2225,1225.000000,36 months,7.43%,69.14,A,A2,< 1 year,...,69.14,NaN,0.0,1,INDIVIDUAL,0,0.0,0,NaN,0.0


In [93]:
lc.drop(['next_pymnt_d'] , axis =1, inplace =True)